In [1]:
!pip install -q transformers torch datasets peft bitsandbytes accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.7 MB/s eta 0:00:00


In [4]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [3]:
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [5]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset_file = "qa_data_cleaned.json"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [7]:
dataset = load_dataset("json", data_files=dataset_file, split="train")
def format_instruction(sample):
    return f"""### Instruction:
{sample['instruction']}

### Response:
{sample['response']}"""
formatted_dataset = dataset.map(lambda sample: {'text': format_instruction(sample)})
print(formatted_dataset[0]['text'])

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [8]:
import json
from datasets import Dataset
dataset_file = "qa_data_cleaned.json"
with open(dataset_file, 'r') as f:
    data = json.load(f)
dataset = Dataset.from_dict({"instruction": [item['instruction'] for item in data],
                             "response": [item['response'] for item in data]})

def format_instruction(sample):
    return f"""### Instruction:
{sample['instruction']}

### Response:
{sample['response']}"""
formatted_dataset = dataset.map(lambda sample: {'text': format_instruction(sample)})
print("Successfully loaded and formatted the dataset.")
print("Sample formatted entry:")
print(formatted_dataset[0]['text'])

Map:   0%|          | 0/321 [00:00<?, ? examples/s]

Successfully loaded and formatted the dataset.
Sample formatted entry:
### Instruction:
How do I undo the most recent local commits in Git?

### Response:
Undo a commit & redo
$ git commit -m "Something terribly misguided" # (0: Your Accident)
$ git reset HEAD~                              # (1)
# === If you just want to undo the commit, stop here! ===
[ edit files as necessary ]                    # (2)
$ git add .                                    # (3)
$ git commit -c ORIG_HEAD                      # (4)

`git reset` is the command responsible for the undo. It will undo your last commit while leaving your working tree (the state of your files on disk) untouched. You'll need to add them again before you can commit them again.
Make corrections to working tree files.
`git add` anything that you want to include in your new commit.
Commit the changes, reusing the old commit message. `reset` copied the old head to `.git/ORIG_HEAD`; `commit` with `-c ORIG_HEAD` will open an editor, which 

In [9]:
from peft import LoraConfig
from transformers import TrainingArguments

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./tinyllama-cmd-adapter", # Directory to save checkpoints
    num_train_epochs=1,                   # As required by the task (one full pass over the data)
    per_device_train_batch_size=4,        # A batch size that fits well on a T4 GPU
    gradient_accumulation_steps=2,        # Simulate a larger batch size (4 * 2 = 8) for stability
    optim="paged_adamw_32bit",            # A memory-efficient optimizer for QLoRA
    logging_steps=25,                     # How often to log the training loss
    learning_rate=2e-4,                   # A standard learning rate for fine-tuning
    fp16=True,                            # Use 16-bit precision for faster training
    report_to="none"                      # Disable reporting to external services
)

In [15]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
    args=training_args,
)

print("Starting fine-tuning... This will take some time.")
trainer.train()
print("Fine-tuning complete!")

trainer.save_model("./tinyllama-cmd-adapter-final")
print("LoRA adapter saved successfully to './tinyllama-cmd-adapter-final'")

Converting train dataset to ChatML:   0%|          | 0/321 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/321 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/321 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3036 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/321 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting fine-tuning... This will take some time.


Step,Training Loss
25,2.121000


Fine-tuning complete!
LoRA adapter saved successfully to './tinyllama-cmd-adapter-final'


In [16]:
!pwd

/content


In [17]:

!zip -r /content/tinyllama-cmd-adapter-final.zip /content/tinyllama-cmd-adapter-final

  adding: content/tinyllama-cmd-adapter-final/ (stored 0%)
  adding: content/tinyllama-cmd-adapter-final/special_tokens_map.json (deflated 79%)
  adding: content/tinyllama-cmd-adapter-final/tokenizer.model (deflated 55%)
  adding: content/tinyllama-cmd-adapter-final/training_args.bin (deflated 51%)
  adding: content/tinyllama-cmd-adapter-final/adapter_config.json (deflated 54%)
  adding: content/tinyllama-cmd-adapter-final/adapter_model.safetensors (deflated 8%)
  adding: content/tinyllama-cmd-adapter-final/chat_template.jinja (deflated 60%)
  adding: content/tinyllama-cmd-adapter-final/README.md (deflated 66%)
  adding: content/tinyllama-cmd-adapter-final/tokenizer_config.json (deflated 69%)
  adding: content/tinyllama-cmd-adapter-final/tokenizer.json (deflated 85%)
